# Good hint Migration 
This notebook demonstrates how an LLM performs when presented with a Spring Security migration issue with a good example

Installing pre requisites and configuring Kai with `GPT-3.5-Turbo`

In [ ]:
%pip uninstall kai -y
%pip install --no-cache-dir git+https://github.com/konveyor/kai.git@main
%pip install python-dotenv

In [2]:
from IPython.display import display, Markdown
from kai.llm_interfacing.model_provider import ModelProvider
from kai.kai_config import KaiConfigModels, SupportedModelProviders
from dotenv import load_dotenv
import os
load_dotenv(override=True) 

# Initialize the model provider
model = ModelProvider.from_config(KaiConfigModels(
    provider=SupportedModelProviders.CHAT_OPENAI,
    args={"model": "gpt-4o-mini"},
))

# Async rendering function for displaying the response
async def rendered_llm_call(prompt: str):
    response = await model.ainvoke_llm(prompt)
    display(Markdown(response.content))
    return response



/Users/sraghuna/local_dev/kai-notebook-solution-server/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Below is the snippet we are trying to migrate from springboot2 to springboot3. This code uses the discouraged `configure(AuthenticationManagerBuilder)` method inside a `WebSecurityConfigurerAdapter`, both of which are deprecated in Spring Security 5.7+ and removed in Spring Boot 3.

Our goal is to assess how well the LLM refactors this configuration without a hint, and whether it can replace deprecated patterns with the right approach.

In [3]:
before_code = """\
import org.springframework.context.annotation.Bean;
import org.springframework.context.annotation.Configuration;
import org.springframework.security.config.annotation.authentication.builders.AuthenticationManagerBuilder;
import org.springframework.security.config.annotation.web.configuration.EnableWebSecurity;
import org.springframework.security.config.annotation.web.configuration.WebSecurityConfigurerAdapter;
import org.springframework.security.crypto.bcrypt.BCryptPasswordEncoder;
import org.springframework.security.crypto.password.PasswordEncoder;

@Configuration
@EnableWebSecurity
public class SecurityConfig extends WebSecurityConfigurerAdapter {
    @Override
    protected void configure(AuthenticationManagerBuilder auth) throws Exception {
        auth.inMemoryAuthentication()
            .withUser("user")
            .password(passwordEncoder().encode("password"))
            .roles("USER");
    }

    @Bean
    public PasswordEncoder passwordEncoder() {
        return new BCryptPasswordEncoder();
    }
}
"""

This rule flags the use of `configure(AuthenticationManagerBuilder)`. In Spring Boot 3, developers should instead use a `UserDetailsService` and `PasswordEncoder` bean for authentication configuration.

In [4]:
hint_to_migrate="""\
    
  description: configure(AuthenticationManagerBuilder) override is discouraged since Spring Security 5.7.
  message: |
    To migrate your Spring Security configuration:
    1. Remove the `extends WebSecurityConfigurerAdapter` from your configuration class.
    2. Remove the `configure(AuthenticationManagerBuilder)` method.
    3. Define a `UserDetailsService` bean that returns an `InMemoryUserDetailsManager` with your user details.
    4. Define a `PasswordEncoder` bean to handle password encoding.

    """

Similar example showing before and after migration

In [5]:
example_before_code="""\

@Configuration
public class SecurityConfiguration extends WebSecurityConfigurerAdapter {
    @Override
    protected void configure(AuthenticationManagerBuilder auth) throws Exception {
        UserDetails user = User.withDefaultPasswordEncoder()
            .username("user")
            .password("password")
            .roles("USER")
            .build();
        auth.inMemoryAuthentication()
            .withUser(user);
    }
}

"""

example_after_code="""\
@Configuration
public class SecurityConfiguration {
    @Bean
    public InMemoryUserDetailsManager userDetailsService() {
        UserDetails user = User.withDefaultPasswordEncoder()
            .username("user")
            .password("password")
            .roles("USER")
            .build();
        return new InMemoryUserDetailsManager(user);
    }
}
"""



example_before_code_2 = """\
@Configuration
@EnableWebSecurity
public class SecurityConfiguration extends WebSecurityConfigurerAdapter {

    @Override
    protected void configure(AuthenticationManagerBuilder auth) throws Exception {
        auth.inMemoryAuthentication()
            .withUser("testuser")
            .password(passwordEncoderBean().encode("testpass"))
            .roles("USER");
    }

    @Bean
    public PasswordEncoder passwordEncoderBean() {
        return new BCryptPasswordEncoder();
    }
}
"""

example_after_code_2 = """\
@Configuration
@EnableWebSecurity
public class SecurityConfiguration {

    @Bean
    public InMemoryUserDetailsManager inMemoryUserDetailsManager() {
        UserDetails testUser = User.builder()
            .username("testuser")
            .password(passwordEncoderBean().encode("testpass"))
            .roles("USER")
            .build();
        return new InMemoryUserDetailsManager(testUser);
    }

    @Bean
    public PasswordEncoder passwordEncoderBean() {
        return new BCryptPasswordEncoder();
    }
}
"""

example_before_code_3 = """\
@Configuration
@EnableWebSecurity
public class SecurityConfiguration extends WebSecurityConfigurerAdapter {

    @Override
    protected void configure(AuthenticationManagerBuilder auth) throws Exception {
        auth.inMemoryAuthentication()
            .withUser("admin")
            .password(passwordEncoder().encode("adminpass"))
            .roles("ADMIN");
    }

    @Bean
    public PasswordEncoder passwordEncoder() {
        return new BCryptPasswordEncoder();
    }
}
"""

example_after_code_3 = """\
@Configuration
@EnableWebSecurity
public class SecurityConfiguration {

    @Bean
    public InMemoryUserDetailsManager userDetailsService(PasswordEncoder passwordEncoder) {
        UserDetails adminUser = User.builder()
            .username("admin")
            .password(passwordEncoder.encode("adminpass"))
            .roles("ADMIN")
            .build();
        return new InMemoryUserDetailsManager(adminUser);
    }

    @Bean
    public PasswordEncoder passwordEncoder() {
        return new BCryptPasswordEncoder();
    }
}
"""


summary_of_changes="""\
* Removed the WebSecurityConfigurerAdapter class.
* Created a UserDetailsService bean instead of overriding configure(AuthenticationManagerBuilder).
* Defined a SecurityFilterChain bean to handle HTTP security configurations.
* Kept the PasswordEncoder bean for password encoding.

"""

In [6]:

from kai.reactive_codeplanner.agent.reflection_agent import extract_ast_info, Language

original_summary = extract_ast_info(example_before_code, language=Language.Java)
original_summary_2 = extract_ast_info(example_before_code_2, language=Language.Java)
original_summary_3 = extract_ast_info(example_before_code_3, language=Language.Java)
updated_summary = extract_ast_info(example_after_code, language=Language.Java)
updated_summary_2 = extract_ast_info(example_after_code_2, language=Language.Java)
updated_summary_3 = extract_ast_info(example_after_code_3, language=Language.Java)

diff = original_summary.diff(updated_summary)
diff2 = original_summary_2.diff(updated_summary_2)
diff3 = original_summary_3.diff(updated_summary_3)

print(diff)
print(diff2)
print(diff3)

{'classes': {'modified': [{'name': 'SecurityConfiguration', 'super_class': {'old': 'extends WebSecurityConfigurerAdapter', 'new': ''}, 'fields': [], 'methods': {'added': [{'name': 'userDetailsService', 'parameters': '()', 'body': '{\n        UserDetails user = User.withDefaultPasswordEncoder()\n            .username("user")\n            .password("password")\n            .roles("USER")\n            .build();\n        return new InMemoryUserDetailsManager(user);\n    }', 'annotations': [{'name': 'Bean'}]}], 'removed': [{'name': 'configure', 'parameters': '(AuthenticationManagerBuilder auth)', 'body': '{\n        UserDetails user = User.withDefaultPasswordEncoder()\n            .username("user")\n            .password("password")\n            .roles("USER")\n            .build();\n        auth.inMemoryAuthentication()\n            .withUser(user);\n    }', 'annotations': [{'name': 'Override'}]}]}, 'annotations': [{'name': 'Configuration'}]}]}}
{'classes': {'modified': [{'name': 'Security

In [7]:
prompt = f"""
You are assisting with migrating a Spring Boot 2 application to Spring Boot 3.

The code that needs migration is:

```java
{before_code}
```

You are given:

* A diff between a before/after successful **example** migrations.

* A summary of changes applied in a similar migration migration.

* An hint that describes what should change.

Diff (use this as a guiding pattern for migration) - {diff} {diff2} {diff3}

Summary of Changes (from a successful migration) - {summary_of_changes}

Rule Hint - {hint_to_migrate}

Task
Refactor the given code {before_code} to make it compatible with Spring Boot 3 and Spring Security 6.

Ensure deprecated or removed APIs are replaced with valid alternatives.

Output:

Refactored Code in Java

Step-by-Step Migration Process

New Hint Summary: One-paragraph explanation of what changed and why (to guide future similar migrations).

"""

In [8]:

response = await rendered_llm_call(prompt)

### Refactored Code in Java

```java
import org.springframework.context.annotation.Bean;
import org.springframework.context.annotation.Configuration;
import org.springframework.security.config.annotation.web.builders.HttpSecurity;
import org.springframework.security.config.annotation.web.configuration.EnableWebSecurity;
import org.springframework.security.config.annotation.web.configuration.WebSecurityConfigurerAdapter;
import org.springframework.security.crypto.bcrypt.BCryptPasswordEncoder;
import org.springframework.security.crypto.password.PasswordEncoder;
import org.springframework.security.core.userdetails.User;
import org.springframework.security.core.userdetails.UserDetails;
import org.springframework.security.core.userdetails.UserDetailsService;
import org.springframework.security.provisioning.InMemoryUserDetailsManager;

@Configuration
@EnableWebSecurity
public class SecurityConfig {

    @Bean
    public UserDetailsService userDetailsService(PasswordEncoder passwordEncoder) {
        UserDetails user = User.builder()
            .username("user")
            .password(passwordEncoder.encode("password"))
            .roles("USER")
            .build();
        return new InMemoryUserDetailsManager(user);
    }

    @Bean
    public PasswordEncoder passwordEncoder() {
        return new BCryptPasswordEncoder();
    }

    @Bean
    public SecurityFilterChain securityFilterChain(HttpSecurity http) throws Exception {
        http
            .authorizeRequests()
                .anyRequest().authenticated()
                .and()
            .formLogin();
        return http.build();
    }
}
```

### Step-by-Step Migration Process

1. **Remove Inheritance**: The `WebSecurityConfigurerAdapter` class was removed from the configuration class, as it is no longer needed in Spring Security 6.
2. **UserDetailsService Bean**: A `UserDetailsService` bean was created to return an `InMemoryUserDetailsManager` with the user details instead of overriding the `configure(AuthenticationManagerBuilder)` method.
3. **PasswordEncoder Bean**: The `PasswordEncoder` bean was retained for password encoding.
4. **SecurityFilterChain Bean**: A `SecurityFilterChain` bean was defined to handle HTTP security configurations, replacing the previous method of configuring security through the `WebSecurityConfigurerAdapter`.

### New Hint Summary

In migrating from Spring Boot 2 to Spring Boot 3, the primary changes involved removing the `WebSecurityConfigurerAdapter` class and its associated methods, which are now discouraged. Instead, a `UserDetailsService` bean was introduced to manage user details, and a `SecurityFilterChain` bean was defined to handle HTTP security configurations. This approach aligns with the updated practices in Spring Security 6, promoting a more modular and flexible security configuration.

### Evaluation

| Criteria                                                                 | Result         |
|--------------------------------------------------------------------------|----------------|
| Removed deprecated `configure(AuthenticationManagerBuilder)`            |  Yes           |
| Replaced with `UserDetailsService` and `PasswordEncoder` bean           |  Yes |
| Removed `WebSecurityConfigurerAdapter`                                   |  Yes           |
| The output compiles                                                      | <todo>          |
| Preserves behavior (in-memory auth, user with role)                      | Yes        |

In [9]:
import re

# Extract the Java code block from the LLM response
def extract_java_code_from_response(markdown_text: str) -> str:
    code_blocks = re.findall(r"```java(.*?)```", markdown_text, re.DOTALL)
    return code_blocks[0].strip() if code_blocks else ""

# Example usage
refactored_java_code = extract_java_code_from_response(response.content)
print(refactored_java_code)

import org.springframework.context.annotation.Bean;
import org.springframework.context.annotation.Configuration;
import org.springframework.security.config.annotation.web.builders.HttpSecurity;
import org.springframework.security.config.annotation.web.configuration.EnableWebSecurity;
import org.springframework.security.config.annotation.web.configuration.WebSecurityConfigurerAdapter;
import org.springframework.security.crypto.bcrypt.BCryptPasswordEncoder;
import org.springframework.security.crypto.password.PasswordEncoder;
import org.springframework.security.core.userdetails.User;
import org.springframework.security.core.userdetails.UserDetails;
import org.springframework.security.core.userdetails.UserDetailsService;
import org.springframework.security.provisioning.InMemoryUserDetailsManager;

@Configuration
@EnableWebSecurity
public class SecurityConfig {

    @Bean
    public UserDetailsService userDetailsService(PasswordEncoder passwordEncoder) {
        UserDetails user = User.buil

In [10]:
async def improve_hint(model, original_hint, before_code, diff):
    prompt = f"""
You are reviewing a migration rule.

Original Spring Boot 2 code:
```java
{before_code}
```

Migration Diff (JSON format):
```json
{diff}
{diff2}
{diff3}
```

Original Hint:
{original_hint}

Task:

- Analyze the before/after migration example carefully.

- Identify what was missing or vague in the original hint.

- Write an improved hint that:

    - Clearly states what elements should be changed.

    - Suggests correct alternatives.

    - If needed, includes important code patterns, best practices, or notes.


Format your output like:

- Improved Hint: (Print the updated hint with latest improvements)

- Migration Actions:

    - (Step 1)

    - (Step 2)

    - (Step 3)
"""
    result = await model.ainvoke_llm(prompt)
    return result.content.strip()



In [11]:
async def fix_code(model, before_code, refactored_code, improved_hint):
    prompt = f"""
You are assisting in migrating a Spring Boot 2 security config class.

Original code:
```java
{before_code}
```

Refactored attempt:
```java
{refactored_code}
```

Improved migration hint:
{improved_hint}

Task:
- Refactor the code to be valid for Spring Boot 3 and Spring Security 6.
- Remove deprecated APIs. 
- Ensure it uses correct bean definitions, imports, and compiles successfully.

Return only the updated code.
"""
    result = await model.ainvoke_llm(prompt)
    return result.content.strip()


In [12]:
import asyncio

async def get_improved_hint():
    improved = await improve_hint(model, hint_to_migrate, before_code, diff)
    print("Improved Hint:\n", improved)

    updated_code = await fix_code(model, before_code, refactored_java_code, improved)
    print("\n Updated Code:\n", updated_code)

await get_improved_hint()

Improved Hint:
 - Improved Hint: 
  To migrate your Spring Security configuration from Spring Boot 2 to Spring Boot 3, follow these steps carefully:
  
  1. **Remove the `extends WebSecurityConfigurerAdapter`**: This class is no longer needed in Spring Security 5.7 and later.
  
  2. **Remove the `configure(AuthenticationManagerBuilder)` method**: This method is discouraged and should be replaced with a `UserDetailsService` bean.
  
  3. **Define a `UserDetailsService` bean**: Create a method annotated with `@Bean` that returns an `InMemoryUserDetailsManager`. Use the `User.builder()` method to create user details instead of using `inMemoryAuthentication()`.
  
  4. **Define a `PasswordEncoder` bean**: Ensure you have a method that returns a `PasswordEncoder` (e.g., `BCryptPasswordEncoder`) to handle password encoding for your users.
  
  5. **Use `User.withDefaultPasswordEncoder()` cautiously**: This method is convenient for testing but should not be used in production due to security